# MRI analysis pipeline

<img src="pipeline.png" width="900" />

# MRI pre-processing

## Co-ordinate systems
### 3D Cartesian

<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/Coord_system_CA_0.svg" width="400" />

### 3D Spherical

<img src="https://upload.wikimedia.org/wikipedia/commons/4/4f/3D_Spherical.svg" width="400" />

## Inter-subject co-registration

In order to use smallest possible regions of interest, voxels, we would like to put two 3D images from two different subjects on top of each other as well as possible.

In [34]:
# Two images before co-registration
from IPython.display import HTML
HTML("""
<video width="560" height="560" controls loop>
  <source src="human_t1.mov">
</video>
""")

A standard way to do this is to use one of the brain images as a registration target and move the other into its reference frame. This is typically accomplished in two phases: 1) linear, and 2) non-linear co-registration.

### Linear transformations

All of the linear transformations are global, in a way that they affect each voxel the same way, and can be described using a 4x4 transformation matrix,

$$
T =
\begin{pmatrix}
   0.1 & 0   & 0   & -5   \\
   0   & 0.1 & 0   & -6.7 \\
   0   & 0   & 0.1 & -10  \\
   0   & 0   & 0   & 1
\end{pmatrix}
$$

This example matrix describes a space with isotropic voxel size (of 0.1 units) and a translation of -5 -6.7 -10 units (usually in x,y,z coordinates). The linear trasformations can be split into translation, rotation, scaling, shear, and perspective transformations.


Original image

<img src="orig_img.png" width="500" />

Translation

<table><tr>
<td> <img src="trans_img.png" width="500" /> </td>
<td> 
    $$
T_{trans} =
\begin{pmatrix}
   1 & 0 & 30 \\
   0 & 1 & 30 \\
   0 & 0 & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>

Scaling

<table><tr>
<td> <img src="scaled_img.png" width="500" /> </td>
<td> 
$$
T_{scaling} =
\begin{pmatrix}
   1.3 & 0   & 10 \\
   0   & 0.8 & 30 \\
   0   & 0   & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>

Rotation

<table><tr>
<td> <img src="rot_img.png" width="500" /> </td>
<td> 
$$
T_{rot} =
\begin{pmatrix}
   \cos(\theta) & -\sin(\theta) & 30 \\
   \sin(\theta) &  \cos(\theta) & 30 \\
   0            & 0             & 1  
\end{pmatrix}
=
\begin{pmatrix}
   0.980 & -0.199 & 40 \\
   0.199 &  0.980 & 20 \\
   0     &  0     & 1  
\end{pmatrix}
,
$$
$$ \theta = .2 $$
</td>
</tr>
</table>


Shear

<table><tr>
<td> <img src="sheared_img.png" width="500" /> </td>
<td> 
$$
T_{shear} =
\begin{pmatrix}
   1   & 0.1 & 10 \\
   0.1 & 1   & 30 \\
   0   & 0   & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>


Perspective

<table><tr>
<td> <img src="perspective_img.png" width="500" /> </td>
<td> 
$$
T_{perspective} =
\begin{pmatrix}
   1     & 0 & 10 \\
   0     & 1 & 30 \\
   0.001 & 0 & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>


The perspective transformation does not happen in MRI data, so the last row of T is zeros apart from the diagonal 1.

#### How to use the matrices

Each voxel location in the 3D image is represented with a vector in a cartesian coordinate system. The transformed vector is a result of matrix-vector multiplication

\begin{gather}
  v_{old} =
  \begin{pmatrix}
      x\\
      y\\
      z
  \end{pmatrix}
  \\
  v_{new} = T v_{old}
\end{gather}

Several transformations can be applied to the same vector, which may be useful in longitudinal studies, particularly the ones with progressive deformations.

<img src="longitudinal_coregistration.png" width="900" />

$$ v_{new} = T_1 T_2 T_3 T_4 v_{old}$$


#### Additional terminology

Linear co-registration is often divided into two separate parts:

* rigid co-registration, where the object experiences translations and rotations but remains intact otherwise; requiring 6 parameters (3 translation, 3 rotation)
* affine co-registration, using 9 or 12 parameters


### Non-linear co-registration

Non-linear registration is used typically to correct for local deformations after a linear co-registration phase. As unconstrained non-linear transform can fit anything to everything, we need constraints. One way is to assume that deformations are linear in the smaller scale, and use piecewise linear co-registration to correct for deformations. Or, that objects close to each other in the original image should be relatively close in the transformed image, too. The latest is implemented in ANTs software as diffeomorphic mapping.

```shell

antsRegistration --dimensionality 3 --float 0 --output [trnsfrm_nl_,deformed.nii.gz,inverse.nii.gz] --interpolation Linear --winsorize-image-intensities [0.005,0.995] --use-histogram-matching 0 --initial-moving-transform trnsfrm_rigid_0GenericAffine.mat --transform SyN[0.2,2,0] --metric CC[target.nii.gz,orig.nii.gz,1,4] --convergence [400x300x200,1e-5,10] --shrink-factors 4x2x1 --smoothing-sigmas 2x1x0vox --verbose

```

The product of non-linear registration is often a deformation vector field, which describes local deformations of the original image; where to move the voxel center in order to produce the deformed image.

## Co-registration metrics

In order to perform co-registration, we need a way to measure goodness of registration. Correlation 

$$
r =\frac{n \sum{xy} - (\sum{x})(\sum{y})}{\sqrt{\left(n \sum{x^2}-(\sum{x})^2\right)\left(n \sum{y^2}-(\sum{y})^2\right)}}
$$

is one possible choice. The point of corrrelation is to fit a line so that the fit minimizes the sum of distances between the points and the line 


<img src="PNS.png" width="400" />

and the correlation coefficient r tells the goodness of co-registration. A different solution is found by moving one of the images and computing correlation again. With the metric available, the co-registration task can be represented as a minimization problem.


In [32]:
# Two images before co-registration
from IPython.display import HTML
HTML("""
<video width="560" height="560" controls loop>
  <source src="before_coreg.mov">
</video>
""")

<img src="orig_scatter.png" width="600" />

In [33]:
# Two images after co-registration
from IPython.display import HTML
HTML("""
<video width="560" height="560" controls loop>
  <source src="after_coreg.mov">
</video>
""")

<img src="coreg_scatter.png" width="600" />
<img src="both_scatter.png" width="600" />

Correlation metric works for data which correlates positively but may fail with negatively correlating data, such as when co-registering MRI and CT images to each other.

A partial solution to the problem is mutual information metric

$$
\mathrm {I}(X;Y) = \sum_{y \in Y}{\sum_{x \in X}{p_{(X,Y)}(x,y) \log{\left(\frac{p_{(X,Y)}(x,y)}{p_X(x) p_Y(y)}\right)}}}
$$

$$
p_{X,Y}(x,y) = p (X=x\ \mathrm {and} \ Y=y)
$$	

which computes a joint probability distribution 

<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/95/Multivariate_normal_sample.svg" />
</div>

and uses information theoretic approach to find common information in both of the images.

### Interpolation

After moving an image to another location, the image intensity values need to be recomputed in the new location and potentially new co-ordinate system. This process is called interpolation. Some common forms of interpolation are nearest neighbor, linear, quadratic, etc.

In the nearest neighbor interpolation, the new grid voxel gets the value from the nearest image point. In the linear case, the value is a weighted sum of a linear fit using 8 nearest image voxels, where the weight is inversely proportional to the distance.

<img src="linear_interpolation.png" width=500 />

The number of datapoints needed for interpolation rises dramatically with the number of dimensions as well as the complexity of the interpolating function. The internal interpolation has to be carefully chosen in order to get a smooth optimization problem as well as fast execution of the process.


### Additional need for co-registration

One might want to use the transforms the opposite way, for example to move a mask drawn on the target on the individual brain image. The linear transform matrix inverse provides the opposite transform
    
$$ v_{old} = T^{-1} v_{new} $$

In the non-linear mapping, this is not as easy as the deformation field is discrete and does not provide direct opposite mapping for the inverse process. The best available method is to simultaneously estimate the non-linear trasformation both ways.

## Time-series pre-processing

In diffusion MRI (dMRI) and functional MRI (fMRI) we acquire series of 3D volumes.

In dMRI, the contrast comes from diffusion sensitizing gradient-induced differences in the volumes. Signal in a gradient-specified direction is measured, the signal attenuates (disappears) faster in the directions where water molecules can diffuse more. And this gives an orientation selective contrast.


In [28]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls loop>
  <source src="dwi.mov">
</video>
""")

 
In fMRI, the time-related contrast comes from blood oxygen-induced changes that are brought up by energy demand in a specific brain location.


In [29]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls loop>
  <source src="fmri.mov">
</video>
""")

If the series contain subject motion, the voxel time-series do not represent real voxel signal. To correct for motion, we use co-registration tools presented above to transform each 3D volume of the series to the common reference frame. As the brain remains the same throughout the series acquisition rigid co-registration is often feasible for motion correction, except for potential susceptibility-induced artefacts or distance-to-the-receiver-related field inhomogeneities.


In [26]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls>
  <source src="mc_fmri.mov">
</video>
""")

Each 3D fMRI volume is typically acquired as a series of 2D slices. The slice acquisitions are spread over the typical 1-2 seconds, over which the volume is acquired. As the assumption is that the signal is in the same time-point in each volume, this needs to be corrected.

<img src="slice_timing.png" width=900 />

This slice-timing correction is performed with convolution, using timing of the slice acquisition.

        * dMRI specific: eddy current correction
            * gradient changes induce eddy currents that can be corrected with specialiced algorigthms

## Smoothing

Smoothing is performed for several reasons

Some co-registration algorightms work better with smoothed data

Smoothing removes small variations in multi-subject studies

Data becomes more gaussian after smoothing, which helps in statistical inference


In [30]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls>
  <source src="smoothing.mov">
</video>
""")

## The pre-processing pipeline

The pre-processing steps are rather established and there exists plenty of software to perform the tasks reliably enough. Furthermore, the pre-processing steps can be automated with a bit of scripting releasing the researcher from the laborous manual tasks. Plenty of scripts exist that perform these tasks. 

fMRI:

<img src="fMRI_pipeline.png" width="900" />

dMRI:

<img src="dMRI_pipeline.png" width="700" />


An example code in Snakemake formalism performing slice timing and motion correction to an fMRI dataset ../derivatives/{dataset}_EPI_ms.nii.gz, where {dataset} stands for any string. Slice timing is performed using FSL tool slicetimer and motion correction with ANTs tool antsMotionCorr.

```python
rule motion_correction:
    input:
        data="../derivatives/{dataset}_EPI_ms_st.nii.gz",
        reference="../derivatives/{dataset}_EPI_ms_st_reference.nii.gz"
    output:
        "../derivatives/{dataset}_EPI_ms_st_mc.nii.gz"
    params:
        prefix="../derivatives/{dataset}_EPI_st"
    shell:
        "antsMotionCorr --dimensionality 3 --output [{params.prefix}_mc_,{output},{params.prefix}_ref.nii.gz] --transform Rigid[0.1] --metric GC[{input.reference},{input.data},1,32,Regular,0.2] --iterations 50x50 --useFixedReferenceImage 1 --useScalesEstimator 1 --smoothingSigmas 2x0 --shrinkFactors 2x1"

rule slice_timing:
    input:
        "../derivatives/{dataset}_EPI_ms.nii.gz"
    output:
        "../derivatives/{dataset}_EPI_ms_st.nii.gz"
    shell:
        "slicetimer -i {input} -o {output} -r 1 --odd"
```

* show with images & videos
* voxel time-series before and after the process fig
* työkaluja löytyy
    * spesifisiä tai yleisiä
    * osassa graafiset käyttöliittymät
    * lista tähän

## Parametric maps from the data

### Diffusion tensor imaging (DTI)

In [31]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls loop>
  <source src="dwi.mov">
</video>
""")


The diffusion signal from the different gradient-weighted volumes is fitted into the 6-parameter exponential decay model

$$ S(g,b) = S_0 e^{-b g^T Dg }, $$

where b is the b-value, and g is the gradient direction, and

$$
D =
\begin{pmatrix}
   D_{xx} & D_{xy} & D_{xz} \\
   D_{yx} & D_{yy} & D_{yz} \\
   D_{zx} & D_{zy} & D_{zz} 
\end{pmatrix}
$$
is the diffusion tensor, which can be visualized as an ellipsoid

<img src="ellipsoid.png" width="400" />

where the eigenvalues of the rank-2 matrix can be used to compute diffusivity maps, such as Fractional Anisotropy FA

$$ \mathrm {FA} = \sqrt{\frac{1}{2} \frac{(\lambda_1-\lambda_2)^2 + (\lambda_1-\lambda_3)^2 + (\lambda_2-\lambda_3)^2}{\lambda_1^2 + \lambda_2^2 + \lambda_3^2}} $$

<img src="dwi_FA.png" width="650" />

or Mean Diffusivity

$$ \mathrm{MD} = (\lambda_1 + \lambda_2 + \lambda_3)/3 $$

<img src="dwi_MD.png" width="650" />


    * eigen decomposition -> eigenvalues and vectors
    * eigenvalues -> parametric maps FA, MD show these

